# Design of a Folded-Cascode Op Amp

The following is a walk-through of an example in the Philip Allen book.

## Specification

We are tasked with designing a folded-cascode op-amp of the following topology:

![Folded Cascode Op Amp](images/1_foldedcascode.PNG "Folded Cascode Op Amp")

We are given the following specification:

| Name                              | Value [V]             | Description |
| --------------------------------- |----------------------:| :-----------|
| Slew Rate                         | 10 $\frac{V}{\mu s}$  |             | 
| Load Capacitance ($C_{L}$         | 10 $pF$               |             | 
| Maximum Output Voltage            | 2 V                   |             | 
| Minimum Output Voltage            | 0.5 V                 |             | 
| Power Supply Voltage ($V_{DD}$    | 2.5 V                 |             | 
| Gain Bandwidth Product (GB)       | 10 MHz                |             | 
| Minimum Input Common Mode Voltage | 1 V                   |             | 
| Maximum Input Common Mode Voltage | 2.5 V                 |             | 
| Differential Voltage Gain         | $\geq 3000$ V/V       |             |  
| Power Dissipation ($P_{diss}$)    | $\leq 5$ mW           |             | 

We are also provided with the following process parameters which we will use in our hand calculations.

| Name                              | Value [V]                  | Description |
| --------------------------------- |---------------------------:| :-----------|
| $K_{N}^{\prime}$                  | 120 $\frac{\mu A}{V^{2}}$  |             | 
| $K_{P}^{\prime}$                  | 25 $\frac{\mu A}{V^{2}}$   |             | 
| $V_{TN}$                          | 0.5 V                      |             |
| $V_{TP}$                          | 0.5 V                      |             |
| $\lambda_{N}$                     | 0.06 1/V                   |             |
| $\lambda_{P}$                     | 0.08 1/V                   |             |
| $L$                               | 0.5 $\mu$ m                |             |

### Slew Rate

Firstly we're directed to use the slew rate in order to get the tail current in the first stage. So $I_{3} = SR \cdot C_{L}$.

In [2]:
import math 

def tail_current_calc(SR, CL):
    tail_current = SR * CL
    return tail_current


SR = 10e6
CL = 10e-12

tail_current = tail_current_calc(SR, CL)
str1 = 'The value of tail current needs to be greater than ' + repr(tail_current) + ' [A], or approximately ' + repr(round(tail_current*1e6, 2)) + ' [uA].'
print(str1)

The value of tail current needs to be greater than 9.999999999999999e-05 [A], or approximately 100.0 [uA].


### Bias Currents in Output Cascodes

Next, we will select our bias currents in the output cascodes, based off our chosen tail current.

They are designed to always be larger than our tail current so that the DC current in the cascode mirror will never go to zero.

We aim to have currents that are 1.2 to 1.5 times the tail current. Thus, we will choose $I_{4} = I_{5} = 125 \mu$A. Now all currents in the circuit are defined.

### Maximum Output Voltage, $V_{out}(max)$

Next we will determine the sizing of M4/M5 and M6/M7 from the maximum output voltage. As the maximum output voltage is 2V and $V_{DD}$ is 2.5 V, we figure that we can split the voltage drop across M5 and M7 evenly.

So, let's say that $V_{SD5(Sat)} = \frac{1}{2}(V_{DD} - V_{out}(max)) = 0.5 \cdot 0.5 = 0.25$V.

Knowing this, then recall that in saturation, $I_{D} = \frac{1}{2} K_{P}^{\prime} S (V_{sd}(sat))^{2}$.

So, $$S_{5} = \frac{2 I_{5}}{K_{P}^{\prime} (V_{sd5}(sat))^{2}}$$


In [5]:
def S5_calc(Kp, Vds5_sat, I5):
    S5 = (2*I5)/(Kp * (Vds5_sat)**2)
    return S5

Kp = 25e-6
I5 = 125e-6
Vds5_sat = 0.25

S5 = S5_calc(Kp, Vds5_sat, I5)
str1 = 'The sizing of S5 is calculated to be ' + repr(S5) + ' [um/um].'
print(str1)

The sizing of S5 is calculated to be 160.0 [um/um].


We take similar sizing for M6 and M7 also. In this case, these transistors will only see 125$\mu$A in a worst case scenario (when all current is flowing in one branch).

So, S4 = S5 = S6 = S7 = 160 $\frac{\mu m}{\mu m}$.

### Minimum Output Voltage, $V_{out}(min)$

We can now get a sizing on M8/M9 and M10/M11 by a similar argument. We are expecting a minimum output voltage of 0.5 V, so again splitting the voltage drop across M9 and M11, we figure that $V_{ds9}(sat) = \frac{1}{2} 0.5 = 0.25$V.

Then, as above, we assume a current of 125 $\mu$A and calculate a sizing as:
$$S_{9} = \frac{2 I_{9}}{K_{N}^{\prime} (V_{ds9}(sat))^{2}}$$

In [6]:
Kn = 120e-6
I9 = 125e-6
Vds9_sat = 0.25

S9 = S5_calc(Kn, Vds9_sat, I9)  # Reuse the function above.
str1 = 'The sizing of S9 is calculated to be ' + repr(S9) + ' [um/um].'
print(str1)

The sizing of S9 is calculated to be 33.333333333333336 [um/um].


We note that this sizing doesn't match the book here. In the book it calculates this sizing to be 20 um/um. If $K^{\prime}_{N} = 200 \frac{\mu A}{V^{2}}$, then that sizing is arrived at.

### GB and M1/M2 sizing

Recall that $GB = \frac{g_{m1}}{C_{L}}$. Now, also recall that $I_{D} = \frac{g_{m}^{2}}{2 K^{\prime}_{N} I_{D}}$. So, as we don't know the transconductance of our input pair, we can re-arrange to get a sizing in terms of our known values:

$$ S1 = \frac{GB^{2} C_{L}^{2} }{K^{\prime}_{N} I_{3}}$$

Note that we have removed our half factor by using the tail current, $I_{3}$ here.

In [13]:
def S1_calc(GB, CL, Kn, I_tail):
    S1 = ((2 * math.pi * GB * CL)**2)/(Kn * I_tail)
    return S1

GB = 10e6

S1 = S1_calc(GB, CL, Kn, tail_current)
str1 = 'The sizing of S1 is calculated to be ' + repr(S1) + ' [um/um].'
print(str1)

The sizing of S1 is calculated to be 32.89868133696452 [um/um].


Thus, we'll take a sizing for S1/S2 of $\approx 33$ [um/um].

### Minimum input CM Voltage

Next we will look to size our tail current transistor by using the restriction on our minimum input CM voltage.
Recall that $ S = \frac{2 I}{K^{\prime} V_{ov}^{2}} $. Now, $V_{ov} = V_{gs} - V_{th} = V_{g} - V_{s} - V_{th}$. In this case, the lowest value on $V_{g}$ is $V_{in}(min)$ and the drop across the transistors M1/M2 is given my $\sqrt{ \frac{ 2 I_{1} }{K^{\prime}_{N} S_{1}}}$. Substituting in, we get the following relationship for the sizing of S3.

$$ S_{3} = \frac{2 I_{3}}{K_{N}^{\prime} ( V_{in}(max) - V_{SS} - \sqrt{I_{3} K^{\prime}_{N} S_{1} } - V_{T1})^{2}}$$




In [20]:
def S3_calc(I3, Kn, Vin_min, Vss, S1, Vtn):
    S3 = (2 * I3)/(Kn * (Vin_min - Vss - math.sqrt((I3/(Kn * S1))) - Vtn)**2)
    return S3

Vin_min = 1
Vss = 0
S1 = 33
Vtn = 0.5
I3 = tail_current

S3 = S3_calc(I3, Kn, Vin_min, Vss, S1, Vtn)
str1 = 'The sizing of S3 is calculated to be ' + repr(S3) + ' [um/um].'
print(str1)

The sizing of S3 is calculated to be 14.325568790245923 [um/um].


So, we choose a sizing on S3 of 15.

### Maximum input CM Voltage

Now we will verify that our previous sizing of the transistors M4 and M5 will satisfy the maximum input common mode voltage.

In a similar manner, recall that $ S = \frac{2 I}{K^{\prime} V_{ov}^{2}} $. So, assuming we're in saturation, we can bound our gate source voltage at $V_{DD} + V_{T1}$. Then our worst case scenario for the source voltage is $V_{in}(max)$.

So, for S4 we get the following relationship:

$$ S_{4} = \frac{2 I_{4}}{K_{P}^{\prime} ( V_{DD} - V_{in}(max) + V_{T1})^{2}}$$

In [23]:
def S4_calc(I4, Kp, Vin_max, Vdd, Vtp):
    S4 = (2 * I4)/(Kp * (Vdd - Vin_max + Vtp)**2)
    return S4

Vin_max = 2.5
Vdd = 2.5
Vtp = 0.5
I4 = I5

S4_check = S4_calc(I4, Kp, Vin_max, Vdd, Vtp)
str1 = 'The sizing of S4 is calculated to be ' + repr(S4_check) + ' [um/um].'
print(str1)

The sizing of S4 is calculated to be 40.0 [um/um].


So, we see that a sizing of 40 [um/um] will suffice. As we previously determined our sizing to be 160 [um/um], based off the current we wanted to flow in these branches, we confirm that this will indeed do the job for us.

So, as $ S_{4} = \frac{2 I_{4}}{K_{P}^{\prime} ( V_{DD} - V_{in}(max) + V_{T1})^{2}}$, thus $ ( V_{DD} - V_{in}(max) + V_{T1})^{2} S_{4} = \frac{2 I_{4}}{K_{P}^{\prime}}$

In [34]:
def max_CMvoltage_calc(I4, Kp, S4, Vtp):
    max_delta_volt = math.sqrt((2*I4)/(Kp * S4))
    max_CMV = Vdd + Vtp - max_delta_volt
    return max_CMV

S4 = 160

max_CMV = max_CMvoltage_calc(I4, Kp, S4, Vtp)
str1 = 'The maximum CM voltage on M4 with a size of ' + repr(S4) + ' is calculated to be ' + repr(max_CMV) + ' [V].'
print(str1)

The maximum CM voltage on M4 with a size of 160 is calculated to be 2.75 [V].


### Power Estimation

Next we estimate our power, based off our expected currents. This gives us a figure of 0.875 mW, so under our aim of 5 mW.



In [46]:
def power_estimate(Vdd, I3, I4, I5):
    p_diss = Vdd * (I3 + I4 + I5)
    return p_diss


p_diss = power_estimate(Vdd, I3, I4, I5)
str1 = 'The power dissipated, is expected to be ' + repr(p_diss) + ' [W], or ' + repr(p_diss*1e3) + ' [mW].'
print(str1)


The power dissipated, is expected to be 0.000875 [W], or 0.875 [mW].


0.000125

### Small-Signal Voltage Gain

Finally, we'd like to confirm that our design's small-signal voltage gain satisfies the specification.

To do so, we'll need to calculate the small signal transconductance and output resistances.

Recall that $g_{m} = \sqrt{2 I_{D} K^{\prime} S }$ and $g_{ds} = \lambda I_{D}$.

In [77]:
def ss_gm(Id, K, S):
    gm = math.sqrt(2 * Id * K * S)
    return gm

def ss_ro(cl_lambda, Id):
    ro = (cl_lambda * Id) ** -1
    return ro

lambda_n = 0.06
lambda_p = 0.08

gm4 = ss_gm(I4, Kp, S4)
rds4 = ss_ro(lambda_p, I4)

gm5 = gm4
rds5 = rds4

str1 = 'S4, S5 : gm is expected to be ' + repr(gm4) + ' [Mho], and rds is expected to be ' + repr(1/rds4) + ' [Mhos].'
print(str1)

I6 = I5-I3/2
S6 = S4
gm6 = ss_gm(I6, Kp, S6)
rds6 = ss_ro(lambda_p, I6)

gm7 = gm6
rds7 = rds6

str2 = 'S6, S7 : gm is expected to be ' + repr(gm6) + ' [Mho], and rds is expected to be ' + repr(1/rds6) + ' [Mhos].'
print(str2)

I8 = I6
S8 = 20
gm8 = ss_gm(I8, Kn, S8)
rds8 = ss_ro(lambda_n, I8)

gm9, gm10, gm11 = gm8, gm8, gm8
rds9, rds10, rds11 = rds8, rds8, rds8

str3 = 'S8, S9, S10, S11 : gm is expected to be ' + repr(gm8) + ' [Mho], and rds is expected to be ' + repr(1/rds8) + ' [Mhos].'
print(str3)

I1 = I3/2
S1= 33
gm1 = ss_gm(I1, Kn, S1)
rds1 = ss_ro(lambda_n, I1)

gm2 = gm1
rds2 = rds1

str4 = 'S1, S2 : gm is expected to be ' + repr(gm1) + ' [Mho], and rds is expected to be ' + repr(1/rds1) + ' [Mhos].'
print(str4)


S4, S5 : gm is expected to be 0.001 [Mho], and rds is expected to be 1e-05 [Mhos].
S6, S7 : gm is expected to be 0.0007745966692414834 [Mho], and rds is expected to be 6.000000000000001e-06 [Mhos].
S8, S9, S10, S11 : gm is expected to be 0.0006000000000000001 [Mho], and rds is expected to be 4.5e-06 [Mhos].
S1, S2 : gm is expected to be 0.0006292853089020909 [Mho], and rds is expected to be 2.9999999999999997e-06 [Mhos].


Building on these, we then calculated the output resistance. Looking downwards, with the cascode, $R_{9} = g_{m9} r_{ds9} r_{ds11}$. Looking upwards, we see $R_{7} = g_{m7} r_{ds7} (r_{ds5} \| r_{ds2})$.

Crunching through the numbers..

In [86]:
R9 = gm9 * rds9 * rds11

print('R9 is calculated to be ' + repr(R9))


def parallel_res(A, B):
    R_tot = (A*B)/(A + B)
    return R_tot

R7 = gm7 * rds7 * (parallel_res(rds5, rds2))

print('R7 is calculated to be ' + repr(R7))

Rout = parallel_res(R9, R7)

print('Rout is calculated to be ' + repr(Rout))

R9 is calculated to be 29629629.62962963
R7 is calculated to be 9930726.528736964
Rout is calculated to be 7437843.780316532


Recalling that $A_{v} = g_{mI} R_{out}$, thus we calculate our gain to be 4680 [V/V], as shown below, which is above our specification of 3000 [V/V].

In [87]:
Av = gm1 * Rout
print('Av is calculated to be ' + repr(Av))

Av is calculated to be 4680.525820861984


## Review

Returning to our specification :

| Name                              | Value Desired [V]     | Value Expected (Hand Calculation) |
| --------------------------------- |----------------------:| :---------------------------------|
| Slew Rate                         | 10 $\frac{V}{\mu s}$  | 10 $\frac{V}{\mu s}$              | 
| Load Capacitance ($C_{L}$         | 10 $pF$               | $\checkmark$                      | 
| Maximum Output Voltage            | 2 V                   | $\checkmark$                      | 
| Minimum Output Voltage            | 0.5 V                 | $\checkmark$                      | 
| Power Supply Voltage ($V_{DD}$    | 2.5 V                 | $\checkmark$                      | 
| Gain Bandwidth Product (GB)       | 10 MHz                | $\checkmark$                      | 
| Minimum Input Common Mode Voltage | 1 V                   | $\checkmark$                      | 
| Maximum Input Common Mode Voltage | 2.5 V                 | $\checkmark$                      | 
| Differential Voltage Gain         | $\geq 3000$ V/V       | 4680 V/V                          |  
| Power Dissipation ($P_{diss}$)    | $\leq 5$ mW           | 0.875 mW                          | 
